# Ming Xie formula

In [ ]:
import sys
sys.path.append('/Users/nneveu/github/zfel')
from zfel.mingxie import mingxie

In [ ]:
# Function signature
help(mingxie)

In [ ]:
# Load and get parameters from elegant files:
top_dir         = '/Users/nneveu/github/beam-shaping/sfg'
dcns_file       = '/joehold/100MeV/elegant_ssnl_100MeV_10mill_de_adjusted_64by512_300emission_steps.txt.out'
#gauss_flat_file = '/jingyi/end_linac_flat_lht/SXRSTART.out' #SXRSTART_flat_laser.h5 # picture doesn't match?
gauss_arb_file  = 'jingyi/end_linac_arb_lht/SXRSTART.out' #SXRSTART_arb_laser.h5

In [ ]:
# Some test parameters
test_params = {    
    'sigma_x':18e-6,
    'und_lambda':0.03,
    'und_k':3.7,
    'current':5000.,
    'gamma':15000/0.511,
    'norm_emit':1e-6,
    'sigma_E':3e6}


mingxie(**test_params) 

# Vectorized version
The function will also work with inputs vectorized as numpy arrays. Here is an example scanning sigma_x

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
n = 20
test_params2 = {}
# Duplicate all values
for k,v in test_params.items():
    test_params2[k] = np.array(n*[v])

# Vary sigma_x 
sx0 = test_params['sigma_x']
sx1 = 2*sx0
test_params2['sigma_x'] = np.linspace(sx0, sx1, n)    

# Calculate
result = mingxie(**test_params2)   
result

In [ ]:
# Plot
fig, ax = plt.subplots()
ax.set_xlabel('sigma_x (um)')
ax.set_ylabel('Gain length (m)')
ax.plot(1e6*test_params2['sigma_x'], result['gain_length'])

# Contour plot

Reproduction of a plot similar to Figure 2 in the originam Ming Xie paper

In [ ]:
i_x =20
i_y = 20
felwave = 1.5e-10
beta = 18
undL = 0.03
K = 3.7
sigmae = 3e6
I = np.linspace(2000, 8000, i_x)
norm_emit = np.linspace(0.5e-6, 2.5e-6, i_y);
gamma=np.sqrt(undL*(1+K**2/2)/2/felwave)
sigma_x = np.sqrt(beta*norm_emit/gamma)

test_params = {    
    'sigma_x':sigma_x[0],
    'und_lambda':undL,
    'und_k':K,
    'current':5000.,
    'gamma':gamma,
    'norm_emit':1e-6,
    'sigma_E':sigmae
}

X,Y = np.meshgrid(I, norm_emit)

contour = np.zeros((i_y, i_x))
for i in range(0, len(sigma_x)):
    test_params['current'] = X[i]
    test_params['norm_emit'] = Y[i]
    test_params['sigma_x'] = sigma_x[i]
    out = mingxie(**test_params)
    contour[i][:] = out['saturation_length']

fig, ax = plt.subplots()
c = ax.contour(X/1000, Y*1e6, contour, 25)
ax.clabel(c, inline=1, fontsize=10, fmt='%1.1f')
ax.set_title('Saturation Length (m)')
ax.set_ylabel('Normalized Emittance (mm-mrad)')
ax.set_xlabel('Current (kA)')